<a id="cell_a"></a>

<p id="title">Cell A</p>
<p>hallodri hallali hallig hallige hallo</p>
<a id="cell_b"></a>

<p id="title">Cell B</p>
<p>hallo halo hall hallihallo halle</p>


In [9]:
import tkinter as tk
from IPython.display import display, HTML
import nbformat
import re


def get_markdown_cells(notebook_path):
    if notebook_path:
        with open(notebook_path) as f:
            notebook_content = nbformat.read(f, as_version=4)
        markdown_contents = []
        for cell in notebook_content.cells:
            if cell.cell_type == 'markdown':
                markdown_contents.append(cell.source)
        
        return markdown_contents
    else:
        return []


def clean(string: str):
    return re.sub(pattern="\\n", repl="", string=string)


class WordSuggestionApp:
    def __init__(self, root, word_list):
        self.word_list = word_list
        self.root = root
        self.root.title("Word Suggestion App")

        # Input Label
        self.label = tk.Label(root, text="Enter letters:")
        self.label.pack(pady=10)

        # Input field
        self.entry = tk.Entry(root, width=40)
        self.entry.pack(pady=10)
        self.entry.bind("<KeyRelease>", self.on_key_release)

        # Frame for suggestion buttons
        self.suggestion_frame = tk.Frame(root)
        self.suggestion_frame.pack(pady=10)

        # List to keep track of suggestion buttons
        self.suggestion_buttons = []
        self.selected_index = -1  # Track the currently selected button

    def on_key_release(self, event):
        input_text = self.entry.get()
        self.update_suggestions(input_text)

    def update_suggestions(self, substring):
        # Clear the current suggestions
        self.clear_suggestions()

        # Find words containing the substring
        filtered_words = [word for word in self.word_list if substring in word]

        # Sort by length and get the 5 shortest
        filtered_words = sorted(filtered_words, key=len)[:5]

        # Create buttons for suggestions
        for index, word in enumerate(filtered_words):
            btn = tk.Button(self.suggestion_frame, text=word, command=lambda w=word: self.select_word(w))
            btn.pack(pady=2, fill='x')  # Fill horizontally
            self.suggestion_buttons.append(btn)

        # Reset selection index
        self.selected_index = -1

    def clear_suggestions(self):
        # Remove all suggestion buttons
        for btn in self.suggestion_buttons:
            btn.destroy()
        self.suggestion_buttons.clear()  # Clear the list of buttons

    def select_word(self, word):
        print(f"Selected word: {word}")
        for i in range(len(word_to_anchor_dict[word])):
            anchor = word_to_anchor_dict[word][i]
            title = word_to_title_dict[word][i]
            link_html = f'<a href="#{anchor}">go to {title}</a>'
            display(HTML(link_html))
        current_text = self.entry.get()  # Get current text in entry
        self.entry.delete(0, tk.END)  # Clear the entry box
        self.entry.insert(0, current_text)

    def navigate_up(self):
        if self.suggestion_buttons and self.selected_index > 0:
            self.selected_index -= 1
            self.highlight_button()

    def navigate_down(self):
        if self.suggestion_buttons and self.selected_index < len(self.suggestion_buttons) - 1:
            self.selected_index += 1
            self.highlight_button()

    def highlight_button(self):
        # Highlight the currently selected button
        for index, btn in enumerate(self.suggestion_buttons):
            if index == self.selected_index:
                btn.config(bg="lightblue")  # Highlight color
            else:
                btn.config(bg="SystemButtonFace")  # Default color

    def handle_keypress(self, event):
        if event.keysym == "Up":
            self.navigate_up()
        elif event.keysym == "Down":
            self.navigate_down()
        elif event.keysym == "Return":
            if self.selected_index != -1:
                self.select_word(self.suggestion_buttons[self.selected_index]["text"])


# parameters
notebook_path = "word_completion.ipynb"


# parse content from markdowns
word_to_anchor_dict = {}
word_to_title_dict = {}
markdown_cells = get_markdown_cells(notebook_path=notebook_path)
for cell in markdown_cells:
    if "</a>" in cell:
        anchor = re.findall(pattern='="(.*?)"', string=cell)[0]
        title = re.findall(pattern='<p id="title">(.*?)</p>', string=cell)[0]
        content = re.findall(pattern='<p>(.*?)</p>', string=cell)[0]
        
        words = list(set([clean(w) for w in content.split(" ") + title.split(" ")]))
        for word in words:
            if word in word_to_anchor_dict:
                word_to_anchor_dict[word] = word_to_anchor_dict[word] + [anchor]
                word_to_title_dict[word] = word_to_title_dict[word] + [title]
            else:
                word_to_anchor_dict[word] = [anchor]
                word_to_title_dict[word] = [title]
# open ui
root = tk.Tk()
app = WordSuggestionApp(root, list(word_to_anchor_dict.keys()))
root.bind("<Key>", app.handle_keypress)
root.mainloop()

Selected word: hallo


Selected word: halle


# Actual Notebook start

<a id="cell_a"></a>

<p id="title">Cell A</p>
<p>hallodri hallali hallig hallige hallo</p>

<a id="cell_b"></a>

<p id="title">Cell B</p>
<p>hallo halo hall hallihallo halle</p>